In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("SparkStreamTrigger")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

22/03/11 22:36:23 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/11 22:36:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/11 22:36:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [1]:
# pre-requistes
# we need a server running on port 9999, that provides text stream to spark
# spark stream is a client, that will connext to a server running on port 9999

# termial
# nc -lk 9999

In [3]:
# Structured Streaming
# Based on Data Frame/ internally there is an RDD
# Micro Batch
# spark batch programming, we use spark.read
# in streaming, we use readStream
# socket is predefined word, 
lineStreamDf = spark\
                .readStream\
                .format("socket")\
                .option("host", "localhost")\
                .option("port", 9999)\
                .load()

# now spark will connect to the server running on port 9999
# then it will wait for the input from server
# user/developer can write something on the server console and Press Entery key
# to send to spark / client

22/03/11 22:36:29 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [4]:
lineStreamDf.printSchema()

root
 |-- value: string (nullable = true)



In [ ]:
# .show() will notwork for stream, it will cause an error
# good for batches, not for stream
# lineStreamDf.show()

In [5]:
# print data on console
# for stream write, we will use .writeStream()
# for batch use .write()
# append is mode
# console is keyword, present console output
# writeStream is an action, which runs the whole show
# reading data, trigger window etc by writeStrema
# in general 1 sec trigger used if not mentioned, default

# trigger time 10 second processingTime
echoOnConsole = lineStreamDf\
                    .writeStream\
                    .outputMode("append")\
                    .format("console")\
                    .trigger(processingTime='10 seconds')\
                    .start()

# start basically start the streaming, listen for input
# as input arrives, it starts processing
# listen for jupyter console for the output

22/03/11 22:38:28 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-529dda04-7d03-4f76-aea0-a623583a0bbd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|Hello 10 seconds ...|
+--------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|wow that seem fas...|
+--------------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|i guess im gettin...|
+--------------------+

-------------------------------------------
Batch: 4
-------------------------------------------
+-------------------+
|              value|
+-------------------+
|not that time i see|
|           whats up|
|                 no|
| 

In [6]:
# query id
echoOnConsole.id

'2bf6cb31-4b4f-4056-8f60-bbf28d69a94f'

In [7]:
# auto generated or user given
echoOnConsole.name

In [8]:
# if any exception
echoOnConsole.exception

<bound method StreamingQuery.exception of <pyspark.sql.streaming.StreamingQuery object at 0x7f9b4030c890>>

In [ ]:
echoOnConsole.recentProgress

In [10]:
# await for termination
# block next operations until any termination
echoOnConsole.awaitTermination()
# this statement will not be executed until stream close
# now we have to go to server and stop the server using Ctrl + C
print("await after")

22/03/11 23:01:30 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999


KeyboardInterrupt: 

In [ ]:
print("next cell")